<a href="https://colab.research.google.com/github/islamicity/VirtualOffice/blob/master/Getting_started_with_BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Before you begin


1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.


### Provide your credentials to the runtime

In [3]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Optional: Enable data table display

Colab includes the ``google.colab.data_table`` package that can be used to display large pandas dataframes as an interactive data table.
It can be enabled with:

In [2]:
%load_ext google.colab.data_table

If you would prefer to return to the classic Pandas dataframe display, you can disable this by running:
```python
%unload_ext google.colab.data_table
```

# Use BigQuery via magics

The `google.cloud.bigquery` library also includes a magic command which runs a query and either displays the result or saves it to a variable as a `DataFrame`.

In [6]:
select * FROM `bigquery-public-data.samples.gsod`


SyntaxError: ignored

In [4]:
# Display query output immediately

%%bigquery --project container-registry-16515
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

,total_rows
0,114420316


In [7]:
# Save output in a variable `df`

%%bigquery --project container-registry-16515 df
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

In [8]:
df

,total_rows
0,114420316


# Use BigQuery through google-cloud-bigquery

See [BigQuery documentation](https://cloud.google.com/bigquery/docs) and [library reference documentation](https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html).

The [GSOD sample table](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&t=gsod&page=table) contains weather information collected by NOAA, such as precipitation amounts and wind speeds from late 1929 to early 2010.


### Declare the Cloud project ID which will be used throughout this notebook

In [9]:
project_id = 'container-registry-16515'

### Sample approximately 2000 random rows

In [21]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

sample_count = 2000
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `bigquery-public-data.samples.gsod`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `bigquery-public-data.samples.gsod`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

print('Full dataset has %d rows' % row_count)
print(row_count)

Full dataset has 114420316 rows
114420316


### Describe the sampled data

In [13]:
df.describe()

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,...,num_mean_station_pressure_samples,mean_visibility,num_mean_visibility_samples,mean_wind_speed,num_mean_wind_speed_samples,max_sustained_wind_speed,max_gust_wind_speed,max_temperature,total_precipitation,snow_depth
count,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1900.000000,1900.000000,1514.000000,...,716.000000,1787.000000,1787.000000,1942.000000,1942.000000,1909.000000,244.000000,1966.000000,1760.000000,115.000000
mean,510410.218385,89979.827324,1986.394109,6.528187,15.644997,52.017369,12.790249,41.713421,12.740000,1014.882166,...,12.276536,12.240403,12.507555,6.801339,12.812049,12.049555,24.116393,43.668667,0.083705,9.656522
std,298534.457868,26753.482124,16.343185,3.462538,8.835467,25.045329,7.877310,23.570849,7.885421,9.574146,...,7.869321,9.216603,7.760289,5.132681,7.878273,6.951540,8.482869,25.037234,0.403187,9.115989
min,10010.000000,6.000000,1930.000000,1.000000,1.000000,-95.800003,4.000000,-64.099998,4.000000,953.299988,...,4.000000,0.000000,4.000000,0.000000,4.000000,1.000000,7.800000,-102.599998,0.000000,0.400000
25%,249080.000000,99999.000000,1976.000000,3.000000,8.000000,38.700001,7.000000,29.299999,6.000000,1009.299988,...,6.000000,6.400000,6.000000,3.300000,7.000000,7.800000,18.100000,30.400000,0.000000,2.800000
50%,527870.000000,99999.000000,1988.000000,7.000000,15.000000,55.299999,8.000000,44.450001,8.000000,1014.599976,...,8.000000,9.600000,8.000000,5.800000,8.000000,11.100000,22.900000,46.599998,0.000000,7.500000
75%,725196.000000,99999.000000,2000.000000,10.000000,23.000000,71.500000,23.000000,58.299999,23.000000,1020.000000,...,23.000000,15.000000,23.000000,8.800000,23.000000,15.000000,28.000000,62.099998,0.010000,11.800000
max,999999.000000,99999.000000,2010.000000,12.000000,31.000000,98.800003,24.000000,80.099998,24.000000,1053.800049,...,24.000000,99.400002,24.000000,44.599998,24.000000,73.800003,62.000000,89.599998,7.870000,55.099998


### View the first 10 rows

In [14]:
df.head(10)

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,...,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
0,30260,99999,1938,12,25,43.299999,6,NaN,NaN,1022.099976,...,None,None,NaN,NaN,False,False,False,False,False,False
1,999999,24026,1949,11,15,38.200001,24,28.200001,24.0,1018.299988,...,None,None,0.00,NaN,False,False,False,False,False,False
2,999999,24057,1959,7,12,68.800003,24,37.500000,24.0,1014.500000,...,None,None,0.00,NaN,False,False,False,False,False,False
3,999999,14942,1960,12,5,44.599998,24,37.200001,24.0,1007.799988,...,None,None,0.00,NaN,False,False,False,False,False,False
4,101470,99999,1965,11,19,23.200001,8,20.500000,8.0,1003.400024,...,None,None,0.02,NaN,False,False,False,False,False,False
5,722655,23005,1974,1,15,58.900002,16,20.000000,16.0,1020.099976,...,None,None,0.00,NaN,False,False,False,False,False,False
6,477380,99999,1982,9,27,78.099998,11,67.199997,11.0,NaN,...,None,None,0.00,NaN,False,False,False,False,False,False
7,315480,99999,1985,11,30,-22.299999,4,-29.900000,4.0,1016.200012,...,None,None,0.00,11.0,False,False,False,False,False,False
8,966070,99999,1985,5,20,83.099998,4,75.300003,4.0,1008.400024,...,None,None,0.00,NaN,True,True,True,True,True,True
9,646540,99999,1985,7,27,77.400002,5,69.500000,5.0,1010.400024,...,None,None,0.00,NaN,False,False,False,False,False,False


In [15]:
# 10 highest total_precipitation samples
df.sort_values('total_precipitation', ascending=False).head(10)[['station_number', 'year', 'month', 'day', 'total_precipitation']]

,station_number,year,month,day,total_precipitation
1525,614150,1975,9,5,7.87
1920,478350,1996,7,18,6.93
336,594780,1987,7,30,6.34
144,786410,1987,6,6,4.33
1328,717750,2007,3,12,3.13
348,320930,1959,7,22,2.95
197,485500,1989,11,4,2.95
652,722110,2004,8,2,2.63
668,916520,1975,12,15,2.56
1789,627600,1991,10,30,2.44


# Use BigQuery through pandas-gbq

The `pandas-gbq` library is a community led project by the pandas community. It covers basic functionality, such as writing a DataFrame to BigQuery and running a query, but as a third-party library it may not handle all BigQuery features or use cases.

[Pandas GBQ Documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_gbq.html)

In [16]:
import pandas as pd

sample_count = 2000
df = pd.io.gbq.read_gbq('''
  SELECT name, SUM(number) as count
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE state = 'TX'
  GROUP BY name
  ORDER BY count DESC
  LIMIT 100
''', project_id=project_id, dialect='standard')

df.head()

,name,count
0,James,272793
1,John,235139
2,Michael,225320
3,Robert,220399
4,David,219028


# Syntax highlighting
`google.colab.syntax` can be used to add syntax highlighting to any Python string literals which are used in a query later.

In [17]:
from google.colab import syntax
query = syntax.sql('''
SELECT
  COUNT(*) as total_rows
FROM
  `bigquery-public-data.samples.gsod`
''')

pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')

,total_rows
0,114420316
